# TS-CAN: rPPG Estimation

This notebook demonstrates the use of the TS-CAN model for remote photoplethysmography (rPPG) estimation. The model is based on the paper [Multi-Task Temporal Shift Attention Networks for On-Device Contactless Vitals Measurement](https://arxiv.org/abs/2006.03790) by Xin Liu and Josh Fromm. The model is implemented in PyTorch.

The rPPG signal is later used to estimate the respiratory rate of the subject.

In [ ]:
"""Temporal Shift Convolutional Attention Network (TS-CAN).
Multi-Task Temporal Shift Attention Networks for On-Device Contactless Vitals Measurement
NeurIPS, 2020
Xin Liu, Josh Fromm, Shwetak Patel, Daniel McDuff
"""

import torch
import torch.nn as nn


class Attention_mask(nn.Module):
    def __init__(self):
        super(Attention_mask, self).__init__()

    def forward(self, x):
        xsum = torch.sum(x, dim=2, keepdim=True)
        xsum = torch.sum(xsum, dim=3, keepdim=True)
        xshape = tuple(x.size())
        return x / xsum * xshape[2] * xshape[3] * 0.5

    def get_config(self):
        """May be generated manually. """
        config = super(Attention_mask, self).get_config()
        return config


class TSM(nn.Module):
    def __init__(self, n_segment=10, fold_div=3):
        super(TSM, self).__init__()
        self.n_segment = n_segment
        self.fold_div = fold_div

    def forward(self, x):
        nt, c, h, w = x.size()
        n_batch = nt // self.n_segment
        x = x.view(n_batch, self.n_segment, c, h, w)
        fold = c // self.fold_div
        out = torch.zeros_like(x)
        out[:, :-1, :fold] = x[:, 1:, :fold]  # shift left
        out[:, 1:, fold: 2 * fold] = x[:, :-1, fold: 2 * fold]  # shift right
        out[:, :, 2 * fold:] = x[:, :, 2 * fold:]  # not shift
        return out.view(nt, c, h, w)


class TSCAN(nn.Module):

    def __init__(self, in_channels=3, nb_filters1=32, nb_filters2=64, kernel_size=3, dropout_rate1=0.25,
                 dropout_rate2=0.5, pool_size=(2, 2), nb_dense=128, frame_depth=20, img_size=36):
        """Definition of TS_CAN.
        Args:
          in_channels: the number of input channel. Default: 3
          frame_depth: the number of frame (window size) used in temport shift. Default: 20
          img_size: height/width of each frame. Default: 36.
        Returns:
          TS_CAN model.
        """
        super(TSCAN, self).__init__()
        self.in_channels = in_channels
        self.kernel_size = kernel_size
        self.dropout_rate1 = dropout_rate1
        self.dropout_rate2 = dropout_rate2
        self.pool_size = pool_size
        self.nb_filters1 = nb_filters1
        self.nb_filters2 = nb_filters2
        self.nb_dense = nb_dense
        # TSM layers
        self.TSM_1 = TSM(n_segment=frame_depth)
        self.TSM_2 = TSM(n_segment=frame_depth)
        self.TSM_3 = TSM(n_segment=frame_depth)
        self.TSM_4 = TSM(n_segment=frame_depth)
        # Motion branch convs
        self.motion_conv1 = nn.Conv2d(self.in_channels, self.nb_filters1, kernel_size=self.kernel_size, padding=(1, 1),
                                      bias=True)
        self.motion_conv2 = nn.Conv2d(
            self.nb_filters1, self.nb_filters1, kernel_size=self.kernel_size, bias=True)
        self.motion_conv3 = nn.Conv2d(self.nb_filters1, self.nb_filters2, kernel_size=self.kernel_size, padding=(1, 1),
                                      bias=True)
        self.motion_conv4 = nn.Conv2d(
            self.nb_filters2, self.nb_filters2, kernel_size=self.kernel_size, bias=True)
        # Apperance branch convs
        self.apperance_conv1 = nn.Conv2d(self.in_channels, self.nb_filters1, kernel_size=self.kernel_size,
                                         padding=(1, 1), bias=True)
        self.apperance_conv2 = nn.Conv2d(
            self.nb_filters1, self.nb_filters1, kernel_size=self.kernel_size, bias=True)
        self.apperance_conv3 = nn.Conv2d(self.nb_filters1, self.nb_filters2, kernel_size=self.kernel_size,
                                         padding=(1, 1), bias=True)
        self.apperance_conv4 = nn.Conv2d(
            self.nb_filters2, self.nb_filters2, kernel_size=self.kernel_size, bias=True)
        # Attention layers
        self.apperance_att_conv1 = nn.Conv2d(
            self.nb_filters1, 1, kernel_size=1, padding=(0, 0), bias=True)
        self.attn_mask_1 = Attention_mask()
        self.apperance_att_conv2 = nn.Conv2d(
            self.nb_filters2, 1, kernel_size=1, padding=(0, 0), bias=True)
        self.attn_mask_2 = Attention_mask()
        # Avg pooling
        self.avg_pooling_1 = nn.AvgPool2d(self.pool_size)
        self.avg_pooling_2 = nn.AvgPool2d(self.pool_size)
        self.avg_pooling_3 = nn.AvgPool2d(self.pool_size)
        # Dropout layers
        self.dropout_1 = nn.Dropout(self.dropout_rate1)
        self.dropout_2 = nn.Dropout(self.dropout_rate1)
        self.dropout_3 = nn.Dropout(self.dropout_rate1)
        self.dropout_4 = nn.Dropout(self.dropout_rate2)
        # Dense layers
        if img_size == 36:
            self.final_dense_1 = nn.Linear(3136, self.nb_dense, bias=True)
        elif img_size == 72:
            self.final_dense_1 = nn.Linear(16384, self.nb_dense, bias=True)
        elif img_size == 96:
            self.final_dense_1 = nn.Linear(30976, self.nb_dense, bias=True)
        elif img_size == 128:
            self.final_dense_1 = nn.Linear(57600, self.nb_dense, bias=True)
        else:
            raise Exception('Unsupported image size')
        self.final_dense_2 = nn.Linear(self.nb_dense, 1, bias=True)

    def forward(self, inputs):
        diff_input = inputs[:, :3, :, :]
        raw_input = inputs[:, 3:, :, :]

        diff_input = self.TSM_1(diff_input)
        d1 = torch.tanh(self.motion_conv1(diff_input))
        d1 = self.TSM_2(d1)
        d2 = torch.tanh(self.motion_conv2(d1))

        r1 = torch.tanh(self.apperance_conv1(raw_input))
        r2 = torch.tanh(self.apperance_conv2(r1))

        g1 = torch.sigmoid(self.apperance_att_conv1(r2))
        g1 = self.attn_mask_1(g1)
        gated1 = d2 * g1

        d3 = self.avg_pooling_1(gated1)
        d4 = self.dropout_1(d3)

        r3 = self.avg_pooling_2(r2)
        r4 = self.dropout_2(r3)

        d4 = self.TSM_3(d4)
        d5 = torch.tanh(self.motion_conv3(d4))
        d5 = self.TSM_4(d5)
        d6 = torch.tanh(self.motion_conv4(d5))

        r5 = torch.tanh(self.apperance_conv3(r4))
        r6 = torch.tanh(self.apperance_conv4(r5))

        g2 = torch.sigmoid(self.apperance_att_conv2(r6))
        g2 = self.attn_mask_2(g2)
        gated2 = d6 * g2

        d7 = self.avg_pooling_3(gated2)
        d8 = self.dropout_3(d7)
        d9 = d8.reshape(d8.size(0), -1)
        d10 = torch.tanh(self.final_dense_1(d9))
        d11 = self.dropout_4(d10)
        out = self.final_dense_2(d11)

        return out

## Load the DeepPhys model

In [ ]:
import respiration.utils as utils

device = torch.device('cpu')

dim = 72

model = TSCAN(img_size=dim)
model = torch.nn.DataParallel(model).to(device)

model_path = utils.file_path('data', 'rPPG-Toolbox', 'BP4D_PseudoLabel_TSCAN.pth')
model.load_state_dict(torch.load(model_path, map_location=device))

model = model.module.to(device)

## Load the test video

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

subject = 'Proband21'
setting = '101_natural_lighting'

frames, meta = dataset.get_video_rgb(subject, setting, show_progress=True)

pleth = dataset.get_vital_sign(subject, setting, utils.VitalSigns.pleth)
breath = dataset.get_vital_sign(subject, setting, utils.VitalSigns.thorax_abdomen)

In [ ]:
import respiration.extractor.mtts_can.preprocess as preprocess

raw, diff = preprocess.preprocess_video_frames(frames, dim)

frame_depth = 20

# The model expects a number of frames that is a multiple of frame_depth
cut_off = (raw.shape[0] // frame_depth) * frame_depth
input_resized = raw[:cut_off]
input_normalized = diff[:cut_off]

# Permute from (T, H, W, C) to (T, C, H, W)
input_resized = torch.tensor(input_resized).permute(0, 3, 1, 2)
input_normalized = torch.tensor(input_normalized).permute(0, 3, 1, 2)

print(input_resized.shape, input_normalized.shape)

# Stack the two channels
frames_chunk = torch.cat((input_normalized, input_resized), dim=1).to(device)
frames_chunk.shape

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(20, 6))

# Permute from (C, H, W) to (H, W, C)
diff_frame = input_normalized[0].permute(1, 2, 0)
raw_frame = input_resized[0].permute(1, 2, 0)

# Normalize the frames
diff_frame = (diff_frame - diff_frame.min()) / (diff_frame.max() - diff_frame.min())
raw_frame = (raw_frame - raw_frame.min()) / (raw_frame.max() - raw_frame.min())

axs[0].imshow(diff_frame)
axs[0].set_title('Diff channel')

axs[1].imshow(raw_frame)
axs[1].set_title('Raw channel')

plt.show()

In [ ]:
with torch.no_grad():
    output = model(frames_chunk)

prediction = output.cpu().detach().numpy().squeeze()
prediction.shape

## Compare the predicted rPPG signal with the ground truth

In [ ]:
import respiration.analysis as analysis

comparator = analysis.SignalComparator(
    prediction,
    breath[1:len(prediction) + 1],
    sample_rate=meta.fps,
    lowpass=0.08,
    highpass=0.5,
    detrend_tarvainen=False,
    filter_signal=True,
    normalize_signal=True,
)

In [ ]:
comparator.errors()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 6))

# Plot the predicted rPPG signal
plt.plot(comparator.prediction, label='Predicted rPPG signal')
plt.plot(comparator.ground_truth, label='Ground truth pleth signal')

plt.title('rPPG estimation using DeepPhys')
plt.xlabel('Frame')
plt.ylabel('Signal value')

plt.legend()
plt.show()